In [33]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [27]:
llm = ChatOllama(model="llama3.2")

In [28]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description="Detailed feedback for the eassy")
    score: int = Field(description="Score out of 10", ge=0, le=10)
    

In [29]:
structured_llm=llm.with_structured_output(EvaluationSchema)

In [30]:
essay="""قائداعظم محمد علی جناحؒ

قائداعظم محمد علی جناحؒ برصغیر کے مسلمانوں کے عظیم رہنما اور پاکستان کے بانی تھے۔ آپ نے نہ صرف مسلمانوں کو ایک علیحدہ قوم کی حیثیت دلائی بلکہ ایک آزاد اور خودمختار ریاست پاکستان کے قیام کو ممکن بنایا۔ آپ کی ولادت 25 دسمبر 1876ء کو کراچی میں ہوئی۔ آپ کے والد کا نام پونجا جناح اور والدہ کا نام مٹھی بائی تھا۔ ابتدائی تعلیم کراچی سے حاصل کی اور اعلیٰ تعلیم کے لیے انگلینڈ چلے گئے جہاں سے آپ نے قانون کی ڈگری حاصل کی۔

محمد علی جناحؒ نے عملی زندگی کا آغاز وکالت سے کیا اور اپنی محنت، قابلیت اور دیانتداری سے بہت جلد کامیاب وکیل بن گئے۔ سیاست میں آپ نے ابتدا میں کانگریس میں شمولیت اختیار کی، کیونکہ اس وقت مسلمانوں اور ہندوؤں کے درمیان اتحاد کی امید تھی۔ لیکن وقت گزرنے کے ساتھ ساتھ جناحؒ کو اندازہ ہوا کہ ہندو اکثریت مسلمانوں کو دبا رہی ہے اور ان کے حقوق کو نظر انداز کیا جا رہا ہے۔ اس حقیقت کے بعد آپ نے آل انڈیا مسلم لیگ میں شمولیت اختیار کی اور مسلمانوں کے حقوق کے تحفظ کے لیے جدوجہد شروع کی۔

قائداعظم نے دو قومی نظریہ کو بنیاد بنا کر مسلمانوں کو ایک علیحدہ قوم قرار دیا اور اس بات پر زور دیا کہ مسلمانوں کا دین، ثقافت، تہذیب، اور طرزِ زندگی ہندوؤں سے مختلف ہے، اس لیے ان کا ایک علیحدہ ملک ہونا چاہیے۔ آپ نے اپنی پرامن، جمہوری اور اصولی سیاست کے ذریعے مسلمانوں کو بیدار کیا اور انہیں ایک پلیٹ فارم پر جمع کیا۔ آپ کی قیادت میں 23 مارچ 1940ء کو قراردادِ پاکستان منظور ہوئی جس نے قیام پاکستان کی راہ ہموار کی۔

قائداعظم کا مشہور قول ہے:
"اتحاد، یقین، قربانی" — یہ ان کی سیاست اور جدوجہد کا نچوڑ تھا۔ آپ نے برصغیر کے مسلمانوں کو ایک منزل دی اور بالآخر 14 اگست 1947ء کو پاکستان دنیا کے نقشے پر ایک آزاد مسلم ریاست کے طور پر ابھرا۔

بدقسمتی سے آزادی کے صرف ایک سال بعد، 11 ستمبر 1948ء کو قائداعظم اس دنیا سے رخصت ہو گئے۔ لیکن ان کی جدوجہد، قربانیاں اور اصول آج بھی ہمارے لیے مشعل راہ ہیں۔ اگر ہم ان کے نقشِ قدم پر چلیں تو پاکستان کو ایک مضبوط، ترقی یافتہ اور خوشحال ملک بنا سکتے ہیں۔

نتیجہ:
قائداعظم محمد علی جناحؒ نہ صرف ایک عظیم رہنما بلکہ ایک مثالی انسان تھے۔ انہوں نے جو خواب دیکھا تھا وہ ہم سب کی ذمہ داری ہے کہ اسے حقیقت کا روپ دیں۔ ہمیں چاہیے کہ ان کے اقوال اور اصولوں پر عمل کرتے ہوئے پاکستان کو ایک فلاحی ریاست بنائیں۔"""

In [31]:
prompt= f'Evalute the language quality of the following eassy and provide a feedback and assign a score out of 10 \n {essay}'
structured_llm.invoke(prompt)

EvaluationSchema(feedback='Language Quality Score: 8/10', score=8)

In [34]:
class EssayState(TypedDict):

    essay: str
    language_feedback:str
    analytic_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [41]:
def evaluate_lang(state:EssayState)->EssayState:
    essay=state['essay']
    prompt= f'Evalute the language quality of the following eassy and provide a feedback and assign a score out of 10 \n {essay}'
    output=structured_llm.invoke(prompt)

    return {"language_feedback":output.feedback,"individual_scores":[output.score] }

In [42]:
def clarity_of_thoughts(state:EssayState)->EssayState:
    essay=state['essay']
    prompt= f'Evalute the clearity of thoughts of the following eassy and provide a feedback and assign a score out of 10 \n {essay}'
    output=structured_llm.invoke(prompt)

    return {"clarity_feedback":output.feedback,"individual_scores":[output.score] }

In [43]:
def analytic_thoughts(state:EssayState)->EssayState:
    essay=state['essay']
    prompt= f'Evalute the depth of analysis of the following eassy and provide a feedback and assign a score out of 10 \n {essay}'
    output=structured_llm.invoke(prompt)

    return {"analytic_feedback":output.feedback,"individual_scores":[output.score] }

In [44]:
def final_evalution(state:EssayState)->EssayState:
    clarity_feedback=state['clarity_feedback']
    language_feedback=state['language_feedback']
    analytic_feedback=state['analytic_feedback']
    all_scores=state['individual_scores']

    prompt= f'Based on the following feedbacks create a summarized feedback \n language feedback - {language_feedback} \n depth of analysis feedback - {analytic_feedback} \n clarity of thought feedback - {clarity_feedback}'
    final_feedback=llm.invoke(prompt).content
    final_score = sum(all_scores)/len(all_scores)
    return {"overall_feedback":final_feedback,"avg_score": final_score}

In [45]:
# Create graph
graph= StateGraph(EssayState)

# careat nodes

graph.add_node("evaluate_lang",evaluate_lang)
graph.add_node("clarity_of_thoughts",clarity_of_thoughts)
graph.add_node("analytic_thoughts",analytic_thoughts)
graph.add_node("final_evalution", final_evalution)

# create edges

graph.add_edge(START,"evaluate_lang")
graph.add_edge(START,"clarity_of_thoughts")
graph.add_edge(START,"analytic_thoughts")

graph.add_edge("evaluate_lang", "final_evalution")
graph.add_edge("clarity_of_thoughts", "final_evalution")
graph.add_edge("analytic_thoughts", "final_evalution")

graph.add_edge("final_evalution", END)

# compile grapg

workflow=graph.compile()

In [46]:
initial_state={
"essay":essay
}
workflow.invoke(initial_state)

{'essay': 'قائداعظم محمد علی جناحؒ\n\nقائداعظم محمد علی جناحؒ برصغیر کے مسلمانوں کے عظیم رہنما اور پاکستان کے بانی تھے۔ آپ نے نہ صرف مسلمانوں کو ایک علیحدہ قوم کی حیثیت دلائی بلکہ ایک آزاد اور خودمختار ریاست پاکستان کے قیام کو ممکن بنایا۔ آپ کی ولادت 25 دسمبر 1876ء کو کراچی میں ہوئی۔ آپ کے والد کا نام پونجا جناح اور والدہ کا نام مٹھی بائی تھا۔ ابتدائی تعلیم کراچی سے حاصل کی اور اعلیٰ تعلیم کے لیے انگلینڈ چلے گئے جہاں سے آپ نے قانون کی ڈگری حاصل کی۔\n\nمحمد علی جناحؒ نے عملی زندگی کا آغاز وکالت سے کیا اور اپنی محنت، قابلیت اور دیانتداری سے بہت جلد کامیاب وکیل بن گئے۔ سیاست میں آپ نے ابتدا میں کانگریس میں شمولیت اختیار کی، کیونکہ اس وقت مسلمانوں اور ہندوؤں کے درمیان اتحاد کی امید تھی۔ لیکن وقت گزرنے کے ساتھ ساتھ جناحؒ کو اندازہ ہوا کہ ہندو اکثریت مسلمانوں کو دبا رہی ہے اور ان کے حقوق کو نظر انداز کیا جا رہا ہے۔ اس حقیقت کے بعد آپ نے آل انڈیا مسلم لیگ میں شمولیت اختیار کی اور مسلمانوں کے حقوق کے تحفظ کے لیے جدوجہد شروع کی۔\n\nقائداعظم نے دو قومی نظریہ کو بنیاد بنا کر مسلمانوں کو ایک علیحد